In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Compare Generative AI Models | Gen AI Evaluation SDK
* [Compare Generative AI Models | Gen AI Evaluation SDK Tutorial](https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/compare_generative_ai_models.ipynb#scrollTo=QN61Ug4hLby5)
     

In [1]:
# @title Install Vertex AI Python SDK
%pip install --upgrade --user --quiet google-cloud-aiplatform[evaluation] plotly

In [2]:
# @title Set GCP information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
EXPERIMENT_NAME = "my-eval-task-experiment"  # @param {type:"string"}

In [3]:
# @title Authentication to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

In [4]:
# @title Initialize Vertex AI SDK
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
# @title Import libraries
import inspect
import logging
import random
import string
import warnings

from IPython.display import HTML, Markdown, display
import pandas as pd
import plotly.graph_objects as go

# Main
from vertexai.evaluation import EvalTask, MetricPromptTemplateExamples
from vertexai.generative_models import GenerativeModel, HarmBlockThreshold, HarmCategory

### Library settings

In [ ]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

### Helper functions

In [18]:
def generate_uuid(length: int = 8) -> str:
    """Generate a uuid of a specified length (default=8)."""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


def print_doc(function):
    print(f"{function.__name__}:\n{inspect.getdoc(function)}\n")


def display_eval_report(eval_result, metrics=None):
    """Display the evaluation results."""

    title, summary_metrics, report_df = eval_result
    metrics_df = pd.DataFrame.from_dict(summary_metrics, orient="index").T
    if metrics:
        metrics_df = metrics_df.filter(
            [
                metric
                for metric in metrics_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )
        report_df = report_df.filter(
            [
                metric
                for metric in report_df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    # Display the title with Markdown for emphasis
    display(Markdown(f"## {title}"))

    # Display the metrics DataFrame
    display(Markdown("### Summary Metrics"))
    display(metrics_df)

    # Display the detailed report DataFrame
    display(Markdown("### Report Metrics"))
    display(report_df)


def display_explanations(df, metrics=None, n=1):
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = df.sample(n=n)
    if metrics:
        df = df.filter(
            ["context", "reference", "completed_prompt", "response"]
            + [
                metric
                for metric in df.columns
                if any(selected_metric in metric for selected_metric in metrics)
            ]
        )

    for index, row in df.iterrows():
        for col in df.columns:
            display(HTML(f"<h2>{col}:</h2> <div style='{style}'>{row[col]}</div>"))
        display(HTML("<hr>"))


def plot_radar_plot(eval_results, metrics=None):
    fig = go.Figure()

    for eval_result in eval_results:
        title, summary_metrics, report_df = eval_result

        if metrics:
            summary_metrics = {
                k: summary_metrics[k]
                for k, v in summary_metrics.items()
                if any(selected_metric in k for selected_metric in metrics)
            }

        fig.add_trace(
            go.Scatterpolar(
                r=list(summary_metrics.values()),
                theta=list(summary_metrics.keys()),
                fill="toself",
                name=title,
            )
        )

    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 5])), showlegend=True
    )

    fig.show()


def sample_pairwise_result(eval_result, n=1, metric=None):
    """Display a random row of pairwise metric result with model responses."""
    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"
    df = eval_result.metrics_table
    df = df.sample(n=n)
    for index, row in df.iterrows():
        display(HTML(f"<h2>Prompt:</h2> <div style='{style}'>{row['prompt']}</div>"))
        display(
            HTML(
                f"<h2>Baseline Model Response:</h2> <div style='{style}'>{row['baseline_model_response']}</div>"
            )
        )
        display(
            HTML(
                f"<h2>Candidate Model Response:</h2> <div style='{style}'>{row['response']}</div>"
            )
        )
        display(
            HTML(
                f"<h2>Explanation:</h2> <div style='{style}'>{row[f'{metric}/explanation']}</div>"
            )
        )
        display(
            HTML(
                f"<h2>Winner:</h2> <div style='{style}'>{row[f'{metric}/pairwise_choice']}</div>"
            )
        )
        display(HTML("<hr>"))


def display_pairwise_win_rate(eval_result, metric=None):
    """Display pairwise aggregated metrics"""
    summary_metrics = eval_result.summary_metrics
    candidate_model_win_rate = round(
        summary_metrics[f"{metric}/candidate_model_win_rate"] * 100
    )
    display(
        HTML(
            f"<h3>Win rate: Autorater prefers Candidate Model over Baseline Model {candidate_model_win_rate}% of time.</h3>"
        )
    )

## Compare and Select Generative Models

You can define an `EvalTask` with pointwise metrics and an evaluation dataset, and then conduct a controlled experiment by running the evaluation multiple times with the same setup, but each run utilizing a different model. This allows you to isolate the impact of the model on the results, and ensuring consistent conditions for comparison.

### Define a Dataset

In [8]:
instruction = "Summarize the following article"

context = [
    "To make a classic spaghetti carbonara, start by bringing a large pot of salted water to a boil. While the water is heating up, cook pancetta or guanciale in a skillet with olive oil over medium heat until it's crispy and golden brown. Once the pancetta is done, remove it from the skillet and set it aside. In the same skillet, whisk together eggs, grated Parmesan cheese, and black pepper to make the sauce. When the pasta is cooked al dente, drain it and immediately toss it in the skillet with the egg mixture, adding a splash of the pasta cooking water to create a creamy sauce.",
    "Preparing a perfect risotto requires patience and attention to detail. Begin by heating butter in a large, heavy-bottomed pot over medium heat. Add finely chopped onions and minced garlic to the pot, and cook until they're soft and translucent, about 5 minutes. Next, add Arborio rice to the pot and cook, stirring constantly, until the grains are coated with the butter and begin to toast slightly. Pour in a splash of white wine and cook until it's absorbed. From there, gradually add hot chicken or vegetable broth to the rice, stirring frequently, until the risotto is creamy and the rice is tender with a slight bite.",
    "For a flavorful grilled steak, start by choosing a well-marbled cut of beef like ribeye or New York strip. Season the steak generously with kosher salt and freshly ground black pepper on both sides, pressing the seasoning into the meat. Preheat a grill to high heat and brush the grates with oil to prevent sticking. Place the seasoned steak on the grill and cook for about 4-5 minutes on each side for medium-rare, or adjust the cooking time to your desired level of doneness. Let the steak rest for a few minutes before slicing against the grain and serving.",
    "Creating a creamy homemade tomato soup is a comforting and simple process. Begin by heating olive oil in a large pot over medium heat. Add diced onions and minced garlic to the pot and cook until they're soft and fragrant. Next, add chopped fresh tomatoes, chicken or vegetable broth, and a sprig of fresh basil to the pot. Simmer the soup for about 20-30 minutes, or until the tomatoes are tender and falling apart. Remove the basil sprig and use an immersion blender to puree the soup until smooth. Season with salt and pepper to taste before serving.",
    "To bake a decadent chocolate cake from scratch, start by preheating your oven to 350°F (175°C) and greasing and flouring two 9-inch round cake pans. In a large mixing bowl, cream together softened butter and granulated sugar until light and fluffy. Beat in eggs one at a time, making sure each egg is fully incorporated before adding the next. In a separate bowl, sift together all-purpose flour, cocoa powder, baking powder, baking soda, and salt. Divide the batter evenly between the prepared cake pans and bake for 25-30 minutes, or until a toothpick inserted into the center comes out clean.",
]

reference = [
    "The process of making spaghetti carbonara involves boiling pasta, crisping pancetta or guanciale, whisking together eggs and Parmesan cheese, and tossing everything together to create a creamy sauce.",
    "Preparing risotto entails sautéing onions and garlic, toasting Arborio rice, adding wine and broth gradually, and stirring until creamy and tender.",
    "Grilling a flavorful steak involves seasoning generously, preheating the grill, cooking to desired doneness, and letting it rest before slicing.",
    "Creating homemade tomato soup includes sautéing onions and garlic, simmering with tomatoes and broth, pureeing until smooth, and seasoning to taste.",
    "Baking a decadent chocolate cake requires creaming butter and sugar, beating in eggs and alternating dry ingredients with buttermilk before baking until done.",
]


eval_dataset = pd.DataFrame(
    {
        "context": context,
        "instruction": [instruction] * len(context),
        "reference": reference,
    }
)

eval_dataset

,context,instruction,reference
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...


### Create an Evaluation Task

#### Documentation

Documentation and example usages for the `EvalTask`

In [ ]:
print_doc(EvalTask)


#### Design text prompt template

For more task-specific prompt guidance, see https://cloud.google.com/vertex-ai/generative-ai/docs/text/text-prompts.

In [40]:
prompt_template = "{instruction}. Article: {context}. Summary:"
prompt_template

'{instruction}. Article: {context}. Summary:'

#### Define metrics


In [41]:
metrics = [
    MetricPromptTemplateExamples.Pointwise.TEXT_QUALITY,
    MetricPromptTemplateExamples.Pointwise.FLUENCY,
    MetricPromptTemplateExamples.Pointwise.SAFETY,
    MetricPromptTemplateExamples.Pointwise.VERBOSITY,
]

#### Define EvalTask & Experiment


In [42]:
summarization_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=metrics,
    experiment=EXPERIMENT_NAME,
)

### Compare Gemini-1.5-Pro with Gemini-1.0-Pro

Evaluate **Gemini-1.5-Pro** model and **Gemini-1.0-Pro** model on the text summarization task defined above using Gen AI Eval Service SDK

#### Model settings


In [43]:
generation_config = {
    "max_output_tokens": 128,
    "temperature": 0.4,
}

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

# Gemini-1.5-Pro
gemini_model_15 = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

# Gemini-1.0-Pro
gemini_model_1 = GenerativeModel(
    "gemini-1.0-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

In [44]:
models = {
    "gemini-pro-1-5": gemini_model_15,
    "gemini-pro-1-0": gemini_model_1,
}

#### Running evaluation

In [25]:
eval_results = []
run_id = generate_uuid()

for model_name, model in models.items():
    experiment_run_name = f"eval-{model_name}-{run_id}"

    eval_result = summarization_eval_task.evaluate(
        model=model,
        prompt_template=prompt_template,
        experiment_run_name=experiment_run_name,
    )

    eval_results.append(
        (f"Model {model_name}", eval_result.summary_metrics, eval_result.metrics_table)
    )

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/my-eval-task-experiment-eval-gemini-pro-1-5-t2ljx00s to Experiment: my-eval-task-experiment


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '{instruction}. Article: {context}. Summary:', 'model_name': 'publishers/google/models/gemini-1.5-pro', 'max_output_tokens': 128, 'temperature': 0.4, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.5-pro.
100%|██████████| 5/5 [00:02<00:00,  2.15it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.5-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference too

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/my-eval-task-experiment-eval-gemini-pro-1-0-t2ljx00s to Experiment: my-eval-task-experiment


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '{instruction}. Article: {context}. Summary:', 'model_name': 'publishers/google/models/gemini-1.0-pro', 'max_output_tokens': 128, 'temperature': 0.4, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.0-pro.
100%|██████████| 5/5 [00:02<00:00,  2.34it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.0-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference too

#### Evaluation Report

In [26]:
for eval_result in eval_results:
    display_eval_report(eval_result)

## Model gemini-pro-1-5

### Summary Metrics

,row_count,text_quality/mean,text_quality/std,fluency/mean,fluency/std,safety/mean,safety/std,verbosity/mean,verbosity/std
0,5.0,4.8,0.447214,5.0,0.0,1.0,0.0,0.8,0.447214


### Report Metrics

,context,instruction,reference,prompt,response,text_quality/explanation,text_quality/score,fluency/explanation,fluency/score,safety/explanation,safety/score,verbosity/explanation,verbosity/score
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Summarize the following article. Article: To m...,This article provides a concise guide to makin...,Coherence: The response presents the steps inv...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,The response is safe as it accurately summariz...,1.0,STEP 1 (Completeness): The response covers all...,1.0
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Summarize the following article. Article: Prep...,Making perfect risotto is a labor of love! Sta...,Coherence: The response presents the steps inv...,5.0,STEP 1: Grammar correctness: The response has ...,5.0,"The response is safe, as it does not contain a...",1.0,STEP 1: The response includes all the key step...,1.0
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Summarize the following article. Article: For ...,"To grill a delicious steak, choose a well-marb...",Coherence: The response presents the steps for...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,The response provides a concise and accurate s...,1.0,STEP 1: The response provides all the necessar...,0.0
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Summarize the following article. Article: Crea...,This article provides a simple recipe for crea...,STEP 1: Assessment:\n- Coherence: The response...,5.0,STEP 1: Assess grammar correctness: The respon...,5.0,"The response provides a safe, accurate, and co...",1.0,STEP 1: The response covers all the key steps ...,1.0
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Summarize the following article. Article: To b...,This article provides a concise guide to bakin...,Coherence: The response presents a clear and l...,4.0,STEP 1: Assess grammar correctness: The respon...,5.0,The response is safe. It accurately summarizes...,1.0,STEP 1 (Completeness): The response covers the...,1.0


## Model gemini-pro-1-0

### Summary Metrics

,row_count,text_quality/mean,text_quality/std,fluency/mean,fluency/std,safety/mean,safety/std,verbosity/mean,verbosity/std
0,5.0,3.0,0.707107,2.6,0.547723,1.0,0.0,1.0,0.0


### Report Metrics

,context,instruction,reference,prompt,response,text_quality/explanation,text_quality/score,fluency/explanation,fluency/score,safety/explanation,safety/score,verbosity/explanation,verbosity/score
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Summarize the following article. Article: To m...,## Spaghetti Carbonara Summary:\n\n**Ingredien...,Coherence: The response presents information i...,3.0,STEP 1: Assess grammar correctness: The respon...,2.0,The response is safe. It provides a summary of...,1.0,The response is slightly verbose by including ...,1.0
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Summarize the following article. Article: Prep...,## Risotto Recipe Summary:\n\nThis summary out...,The response has issues with instruction follo...,2.0,STEP 1: Assess grammar correctness: The respon...,3.0,The response is safe as it contains none of th...,1.0,The response is somewhat verbose. While it eff...,1.0
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Summarize the following article. Article: For ...,## Grilling a Flavorful Steak: A Summary\n\nTh...,Coherence: The response is well-organized and ...,4.0,STEP 1: Assess grammar correctness: There is ...,2.0,The response is safe because it provides instr...,1.0,STEP 1: Completeness: The response includes al...,1.0
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Summarize the following article. Article: Crea...,## Creamy Homemade Tomato Soup Summary:\n\nThi...,Coherence: The response is organized and easy ...,3.0,STEP 1: Assess grammar correctness: The respon...,3.0,The response is safe. It contains none of the ...,1.0,The AI response is somewhat verbose. While it ...,1.0
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Summarize the following article. Article: To b...,## Summary of Chocolate Cake Recipe:\n\nThis r...,Coherence: The response presents the steps cle...,3.0,STEP 1: Assess grammar correctness: The respon...,3.0,"The response is safe, as it does not contain h...",1.0,The response is slightly verbose. While the fo...,1.0


### Explanations for model-based metrics

In [45]:
for eval_result in eval_results:
    display_explanations(eval_result[2], metrics=["fluency"])

### Compare Eval Results


In [46]:
plot_radar_plot(
    eval_results,
    metrics=[
        f"{metric}/mean"
        for metric in ["fluency", "text_quality", "safety", "verbosity"]
    ],
)

In [47]:
summarization_eval_task.display_runs()

,experiment_name,run_name,run_type,state,param.HARM_CATEGORY_DANGEROUS_CONTENT,param.model_name,param.HARM_CATEGORY_HATE_SPEECH,param.prompt_template,param.HARM_CATEGORY_HARASSMENT,param.temperature,...,metric.row_count,metric.pairwise_text_quality/baseline_model_win_rate,metric.verbosity/mean,metric.fluency/std,metric.safety/std,metric.text_quality/std,metric.verbosity/std,metric.safety/mean,metric.fluency/mean,metric.text_quality/mean
0,my-eval-task-experiment,47db0e56-c71c-42bf-b59f-0a41f51a06b9,system.ExperimentRun,COMPLETE,BLOCK_NONE,publishers/google/models/gemini-1.5-pro,BLOCK_NONE,{instruction}. Article: {context}. Summary:,BLOCK_NONE,0.4,...,5.0,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,my-eval-task-experiment,eval-gemini-pro-1-0-t2ljx00s,system.ExperimentRun,COMPLETE,BLOCK_NONE,publishers/google/models/gemini-1.0-pro,BLOCK_NONE,{instruction}. Article: {context}. Summary:,BLOCK_NONE,0.4,...,5.0,NaN,1.0,0.547723,0.0,0.707107,0.000000,1.0,2.6,3.000000
2,my-eval-task-experiment,eval-gemini-pro-1-5-t2ljx00s,system.ExperimentRun,COMPLETE,BLOCK_NONE,publishers/google/models/gemini-1.5-pro,BLOCK_NONE,{instruction}. Article: {context}. Summary:,BLOCK_NONE,0.4,...,5.0,NaN,0.8,0.000000,0.0,0.447214,0.447214,1.0,5.0,4.800000
3,my-eval-task-experiment,53ab9075-09e9-4a87-abe3-74db8cbdf3a8,system.ExperimentRun,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,0.115470,NaN,NaN,NaN,1.066667
4,my-eval-task-experiment,987acf22-ba44-47ac-bd39-5f394b7b0409,system.ExperimentRun,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,0.577350,NaN,NaN,NaN,1.333333
5,my-eval-task-experiment,0bf33c25-3658-45a9-afcc-84240f4c1572,system.ExperimentRun,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,0.057735,NaN,NaN,NaN,1.033333
6,my-eval-task-experiment,718426ec-0198-432e-93e6-b455bcdab005,system.ExperimentRun,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000
7,my-eval-task-experiment,5f94393d-9562-4885-b0d8-dbec25b6a4fe,system.ExperimentRun,COMPLETE,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,0.577350,NaN,NaN,NaN,1.666667
8,my-eval-task-experiment,4b6552b9-cd69-42d3-8d02-cf091a66cedb,system.ExperimentRun,FAILED,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,my-eval-task-experiment,bdc715cd-bd2a-425b-a240-845d832c6bbf,system.ExperimentRun,FAILED,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Compare Two Models Side-by-side (SxS)

To directly compare two models, you can define a `PairwiseMetric` within an `EvalTask` run. This approach allows for a head-to-head assessment of the models' performance.

In [30]:
from vertexai.evaluation import PairwiseMetric

In [31]:
# Baseline model for pairwise comparison
baseline_model = GenerativeModel(
    "gemini-1.0-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

# Candidate model for pairwise comparison
candidate_model = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

In [50]:
# @title Create a "Pairwise Text Quality" metric
text_quality_prompt_template = MetricPromptTemplateExamples.get_prompt_template(
    "pairwise_text_quality"
)

print(text_quality_prompt_template)

pairwise_text_quality_metric = PairwiseMetric(
    metric="pairwise_text_quality",
    metric_prompt_template=text_quality_prompt_template,
    baseline_model=baseline_model,
)



# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by two AI models. We will provide you with the user input and a pair of AI-generated responses (Response A and Response B). You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will first judge responses individually, following the Rating Rubric and Evaluation Steps. Then you will give step-by-step explanations for your judgment, compare the results to declare the winner based on the Rating Rubric and Evaluation Steps.

# Evaluation
## Metric Definition
You will be assessing the Text Quality of each model's response, which measures how effectively the text conveys clear, accurate, and engaging information that directly addresses the user's prompt, considering factors like fluency, coherence, relevance, and conciseness.

## Criteria
Coherence: The re

In [51]:
# @title Evaluation tasks
pairwise_text_quality_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[pairwise_text_quality_metric],
    experiment=EXPERIMENT_NAME,
)


In [52]:
prompt_template

'{instruction}. Article: {context}. Summary:'

In [53]:
# @title Specify candidate model for pairwise comparison
pairwise_text_quality_result = pairwise_text_quality_eval_task.evaluate(
    model=candidate_model,
    prompt_template=prompt_template,
)

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/my-eval-task-experiment-bc7aad1f-7ce6-459e-9dcd-6d2935feb003 to Experiment: my-eval-task-experiment


INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': '{instruction}. Article: {context}. Summary:', 'model_name': 'publishers/google/models/gemini-1.5-pro', 'max_output_tokens': 128, 'temperature': 0.4, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Generating a total of 5 responses from Gemini model gemini-1.5-pro.
100%|██████████| 5/5 [00:02<00:00,  2.14it/s]
INFO:vertexai.evaluation._evaluation:All 5 responses are successfully generated from Gemini model gemini-1.5-pro.
INFO:vertexai.evaluation._evaluation:Multithreaded Batch Inference too

In [54]:
display_eval_report(
    (
        "Side-by-side EvalTask",
        pairwise_text_quality_result.summary_metrics,
        pairwise_text_quality_result.metrics_table,
    )
)

## Side-by-side EvalTask

### Summary Metrics

,row_count,pairwise_text_quality/candidate_model_win_rate,pairwise_text_quality/baseline_model_win_rate
0,5.0,0.8,0.2


### Report Metrics

,context,instruction,reference,prompt,response,baseline_model_response,pairwise_text_quality/explanation,pairwise_text_quality/pairwise_choice
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Summarize the following article. Article: To m...,This article provides a concise guide to makin...,## Spaghetti Carbonara Summary:\n\n**Ingredien...,CANDIDATE response is a better summary of the ...,CANDIDATE
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Summarize the following article. Article: Prep...,Making perfect risotto is a labor of love! Sta...,## Risotto Recipe Summary:\n\nThis summary out...,CANDIDATE response provides a concise and accu...,CANDIDATE
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Summarize the following article. Article: For ...,"To grill a delicious steak, choose a well-marb...",## Grilled Steak Summary:\n\n**Ingredients:**\...,CANDIDATE response is a concise and accurate s...,CANDIDATE
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Summarize the following article. Article: Crea...,This short article provides a simple recipe fo...,## Creamy Homemade Tomato Soup Summary:\n\nThi...,BASELINE response is slightly better because i...,BASELINE
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Summarize the following article. Article: To b...,This article provides a concise guide to bakin...,## Baking a Decadent Chocolate Cake from Scrat...,"CANDIDATE response provides a complete, concis...",CANDIDATE


In [55]:
pairwise_text_quality_result.metrics_table

,context,instruction,reference,prompt,response,baseline_model_response,pairwise_text_quality/explanation,pairwise_text_quality/pairwise_choice
0,"To make a classic spaghetti carbonara, start b...",Summarize the following article,The process of making spaghetti carbonara invo...,Summarize the following article. Article: To m...,This article provides a concise guide to makin...,## Spaghetti Carbonara Summary:\n\n**Ingredien...,CANDIDATE response is a better summary of the ...,CANDIDATE
1,Preparing a perfect risotto requires patience ...,Summarize the following article,Preparing risotto entails sautéing onions and ...,Summarize the following article. Article: Prep...,Making perfect risotto is a labor of love! Sta...,## Risotto Recipe Summary:\n\nThis summary out...,CANDIDATE response provides a concise and accu...,CANDIDATE
2,"For a flavorful grilled steak, start by choosi...",Summarize the following article,Grilling a flavorful steak involves seasoning ...,Summarize the following article. Article: For ...,"To grill a delicious steak, choose a well-marb...",## Grilled Steak Summary:\n\n**Ingredients:**\...,CANDIDATE response is a concise and accurate s...,CANDIDATE
3,Creating a creamy homemade tomato soup is a co...,Summarize the following article,Creating homemade tomato soup includes sautéin...,Summarize the following article. Article: Crea...,This short article provides a simple recipe fo...,## Creamy Homemade Tomato Soup Summary:\n\nThi...,BASELINE response is slightly better because i...,BASELINE
4,To bake a decadent chocolate cake from scratch...,Summarize the following article,Baking a decadent chocolate cake requires crea...,Summarize the following article. Article: To b...,This article provides a concise guide to bakin...,## Baking a Decadent Chocolate Cake from Scrat...,"CANDIDATE response provides a complete, concis...",CANDIDATE


In [56]:
sample_pairwise_result(
    pairwise_text_quality_result, metric="pairwise_text_quality", n=1
)

In [57]:
display_pairwise_win_rate(pairwise_text_quality_result, metric="pairwise_text_quality")